In [49]:
using MLBase
using Flux
using Flux: onehotbatch, argmax, crossentropy, throttle
using Plots
using DecisionTree
using Distances
using LIBSVM
using CSV
using DataFrames
using Random

In [50]:
data = CSV.read("Iris.csv", DataFrame)
X = Matrix(data[:,2:5])
irislabels = data[:,6]
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap,irislabels)


150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [51]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at)
        push!(keepids,rowids...)
    end
    return keepids
end
findaccuracy(predictedvals, groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)
assing_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assing_class (generic function with 1 method)

In [52]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)
x_train = X[trainids,:]
y_train = y[trainids]
x_test = X[testids,:]
y_test = y[testids,:]
x_train = transpose(Float32.(x_train))
x_test = transpose(Float32.(x_test))

4×50 transpose(::Matrix{Float32}) with eltype Float32:
 5.4  5.0  5.4  5.8  5.7  5.4  5.4  4.6  …  6.1  6.4  6.9  6.7  6.3  6.2  5.9
 3.9  3.4  3.7  4.0  4.4  3.9  3.4  3.6     2.6  3.1  3.1  3.1  2.5  3.4  3.0
 1.7  1.5  1.5  1.2  1.5  1.3  1.7  1.0     5.6  5.5  5.4  5.6  5.0  5.4  5.1
 0.4  0.2  0.2  0.2  0.4  0.4  0.2  0.2     1.4  1.8  2.1  2.4  1.9  2.3  1.8

In [53]:
y_train = onehotbatch(y_train,unique(y_test))
y_test = onehotbatch(y_test, unique(y_test))

3×50×1 OneHotArray(::Matrix{UInt32}) with eltype Bool:
[:, :, 1] =
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [54]:
data_train = [(x_train, y_train)]
data_test = [(x_test, y_test)]

1-element Vector{Tuple{LinearAlgebra.Transpose{Float32, Matrix{Float32}}, OneHotArrays.OneHotArray{UInt32, 2, 3, Matrix{UInt32}}}}:
 ([5.4 5.0 … 6.2 5.9; 3.9 3.4 … 3.4 3.0; 1.7 1.5 … 5.4 5.1; 0.4 0.2 … 2.3 1.8], [1 1 … 0 0; 0 0 … 0 0; 0 0 … 1 1;;;])

In [64]:
function get_model()
    c = Chain(
        Dense(4,8,relu),
        Dense(8,8,relu),
        Dense(8,3),
        softmax
    )
end

model = get_model()

loss(x,y) = crossentropy(model(x),y)
opt = ADAM()
evalcb = () -> @show(loss(x_train, y_train))
parameters = Flux.params(model)

Params([Float32[-0.19645852 0.565241 -0.1478348 0.6326399; -0.6800285 -0.6623261 -0.086578645 0.25072914; … ; 0.37160817 -0.0009114677 -0.37272355 -0.38547978; 0.6292211 0.21652362 0.38474223 0.4369485], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[-0.33792573 0.38113874 … -0.003458909 0.018772658; -0.1655897 -0.36877948 … -0.47811192 -0.5280431; … ; 0.32733336 -0.5619471 … 0.48255694 0.17787102; -0.22090027 0.0021322712 … 0.20563376 -0.3435873], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[-0.15948991 -0.17896003 … 0.103806056 0.5482595; -0.6916084 0.10576702 … -0.39872247 -0.6693282; -0.6737147 0.33674052 … 0.4037147 -0.6557772], Float32[0.0, 0.0, 0.0]])

In [65]:
for i in 1:400
    Flux.train!(loss, parameters, data_train, opt, cb = throttle(evalcb, 10))
end

loss(x_train, y_train) = 1.3992577f0
loss(x_train, y_train) = 1.3841319f0
loss(x_train, y_train) = 1.3694214f0
loss(x_train, y_train) = 1.3553178f0
loss(x_train, y_train) = 1.3417029f0
loss(x_train, y_train) = 1.3285486f0
loss(x_train, y_train) = 1.3159045f0
loss(x_train, y_train) = 1.3037245f0
loss(x_train, y_train) = 1.2919662f0
loss(x_train, y_train) = 1.2805938f0
loss(x_train, y_train) = 1.2696412f0
loss(x_train, y_train) = 1.2590308f0
loss(x_train, y_train) = 1.2487394f0
loss(x_train, y_train) = 1.2388383f0
loss(x_train, y_train) = 1.2293118f0
loss(x_train, y_train) = 1.2200937f0
loss(x_train, y_train) = 1.2111894f0
loss(x_train, y_train) = 1.2025579f0
loss(x_train, y_train) = 1.1941584f0
loss(x_train, y_train) = 1.1859846f0
loss(x_train, y_train) = 1.1780086f0
loss(x_train, y_train) = 1.1702108f0
loss(x_train, y_train) = 1.1625776f0
loss(x_train, y_train) = 1.1550984f0
loss(x_train, y_train) = 1.1477654f0
loss(x_train, y_train) = 1.1405572f0
loss(x_train, y_train) = 1.1334369f0
l

In [67]:
y = (y_test[:,25])

3-element Vector{Bool}:
 0
 1
 0

In [68]:
pred = (model(x_test[:,25]))

3-element Vector{Float32}:
 0.008662353
 0.5255514
 0.46578628

In [69]:
function loss_all(data_loader)
    sum([loss(x, y) for (x,y) in data_loader]) / length(data_loader) 
end

function acc(data_loader)
    f(x) = Flux.onecold(cpu(x))
    acces = [sum(f(model(x)) .== f(y)) / size(x,2)  for (x,y) in data_loader]
    sum(acces) / length(data_loader)
end

acc (generic function with 1 method)

In [70]:
data_test
@show train_loss = loss_all(data_train)
@show test_loss = loss_all(data_test)
@show train_acc = acc(data_train)
@show test_acc = acc(data_test)


train_loss = loss_all(data_train) = 0.30439714f0
test_loss = loss_all(data_test) = 0.29232547f0
train_acc = acc(data_train) = 0.99
test_acc = acc(data_test) = 0.98


0.98